# Spark Executor Tuning
***How do you decide how many executors you need, and what configuration?***

```bash
spark-submit \
  --class org.example.MySparkApp \ # Main class for Java/Scala applications
  --master yarn \ # Spark master URL (e.g., yarn, local[*])
  --deploy-mode cluster \ # Whether to deploy in client or cluster mode
  --num-executors 5 \
  --executor-memory 2G \ # Memory per executor (e.g., 2G)
  --executor-cores 2 \  # Number of cores per executor
  myapp.jar 100

**3 Main things to decide**
-  No. of executors
-  No. of cores within each
-  RAM of each

***Per Node Leave out 1 Core and 1GB for Hadoop/YARN/OS Daemons***

Let's work with this scenario. 
- You have 5 nodes
- Each node has 12 cores and 48GB RAM (Yup super powerful)
  
### 1. Fat Executors
- Occupies large proportion of resources on a Node
- The no. of executors in this case would be 1 per node. 1 executor on the node will occupy all the 11 cores and 47GB of RAM (we left out 1 core and 1 GB)
- Therefore total no. of executors = 5
- **Pros**
    - Increased task-level parallelism
    - Enhanced data locality - reduces chances of shuffle; reduces network traffic as you don't need to move data
- **Cons**
    - Under utilisation leads to wastage and excessive costs
    - Failure of single node or executor can lead to significant impact. Recovery would be slower
    - Rate of read data from HDFS or write data to HDFS (HDFS Throughput) might suffer. Fat executors can lead to execessive garbage collection (during GC application needs to be paused) 

### 2. Thin Executors
- The opposite of fat executors is thin, they occupy the min resources
- So in each node you would have 11 executors each with 1 core and ~4 GB RAM (47/11).
- Total no. of executors = 11*5 = 55
- **Pros**
    - Increased executor-level parallelism -> Great for processing small jobs
    - Fault tolerant -> since smaller amount of work is done per executor, it can easily restart 
- **Cons**
    - Reduced data locality -> No. of partitions of data within each executor would be limited
    - High network traffic -> needs to bring in data from different locations
 
### 3. Optimal Executors
##### How to Create an Optimal Executor --> 4 Rules
1. Leave out 1 Core and 1 GB of RAM for Hadoop/YARN/OS Daemons per Node
2. Leave out 1 Executor or (1Core, 1GB Ram) YARN Application Manager at Cluster-level
3. General rule of thumb is 3-5 Cores per executors, more than can lead to execessive garbage collection
4. Spare out some memory for Memory Overhead => max(384MB, 10% of executor memory)

- If we follow the above rules and try to create an optimal executor
    - 5 Nodes; 12 Cores; 48 RAM;
    -  Leaving out 1 Core and 1 GB of RAM for Hadoop/YARN/OS Daemons per Node.
    -  Therefore,
        -  total memory = 47*5 = 235GB;
        -  total cores = 11*5 = 55 Cores
    - Leaving out 1Core, 1GB Ram YARN Application Manager at Cluster-level =>
        -  total memory left = 234GB;
        -  total cores left = 54 Cores
    - So let's take 5 Cores (i.e. tasks) per executors
        - total no. of executors = 54/4 ~ 10 Executors with 5 Cores
        - Memory Per Executors = 234/10 ~ 23 GB
    - Leaving out some memory for Memory Overhead => max(384MB, 0.1*23GB) = 2.3GB
        - Actual memory per executor = 23GB - 2.3GB ~ 20GB
    - Total memory per executor = 20 GB
        - Execution memory (60%) = 20 GB × 0.6 = 12 GB (We can assume dynamic allocation)
        - Storage memory (40%) = 20 GB × 0.4 = 8 GB
        - Execution memory per core = 12 GB ÷ 5 cores = 2.4 GB per core

### Why haven't we considered the size of the data?
- While talking about the size of the data we need to talk about the size of the partition
- Because each core can process 1 partition at a time
- taking the optimal executor configuration, we have 4GB Per Core (20/5). So as long as the partition of data is less than 4GB then we should be fine

spark cluster size -- 200 cores and 100 gb RAM
data to be processed --100 gb
give the calculation of spark for driver memory, driver cores, executor memory, overhead memory, number of executo


--- 


**Rule #1: For Hadoop/YARN/OS Daemons**
- Assuming this is a cluster with 10 nodes (200 cores ÷ 20 cores per node = 10 nodes)
- Reserve per node: 1 core and 1 GB
- Total reserved: 10 cores and 10 GB

**Rule #2: For YARN Application Manager**
- Reserve: 1 core and 1 GB

So available resources:
- Cores: 200 - 10 - 1 = 189 cores
- Memory: 100 GB - 10 GB - 1 GB = 89 GB

Now let's configure using remaining rules:

**Rule #3: Use 3-5 cores per executor**
- Let's use 4 (3-5 best practice) cores per executor
- Number of executors = 189 ÷ 4 ≈ 47 executors

**Rule #4: Calculate memory per executor and overhead**
- Available memory per executor before overhead = 89 GB ÷ 47 ≈ 1.89 GB
- Memory overhead = max(384MB, 10% of 1.89 GB) = max(384MB, 189MB) = 384MB
- Final executor memory =tions n thecount for all system reservations

Given your data size of 100 GB, each executor would handle:
100 GB ÷ 47 ≈ 2.13 GB of data
Per core: 2.13 GB ÷ 4 ≈ 532 MB, which down any part of these calculations in more detail?nfigurations based on specific workload characteristics?rs